# 🔍 SVD: Decomposição que Revela o Essencial dos Dados

## Módulo 9: Álgebra Linear para IA

Fala pessoal! 👋 Chegamos no módulo 9 e agora vamos falar de uma das técnicas mais poderosas da álgebra linear: a **Decomposição SVD** (Singular Value Decomposition)!

Tá, mas o que é essa tal de SVD? Imagina que você tem uma foto de 4K ultra detalhada, mas precisa enviá-la pelo WhatsApp sem perder muito da qualidade. Ou então você tem um sistema de recomendação do Netflix que precisa entender os padrões de milhões de usuários. A SVD é como um "detector de padrões essenciais" que consegue pegar o que realmente importa nos seus dados!

**Neste notebook você vai aprender:**
- 🧮 O que é SVD matematicamente (de forma descomplicada!)
- 📸 Como comprimir imagens mantendo a qualidade
- 🎬 Como criar sistemas de recomendação
- 📊 Redução de dimensionalidade na prática
- 🔧 Implementação em Python com NumPy

Bora descobrir o essencial dos dados! 🚀

In [ ]:
# Setup inicial - importando as bibliotecas que vamos usar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
from sklearn.datasets import make_blobs
import pandas as pd
from PIL import Image
import requests
from io import BytesIO

# Configurações para gráficos mais bonitos
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Seed para reprodutibilidade
np.random.seed(42)

print("🔧 Setup completo! Vamos começar a descobrir o essencial dos dados!")

## 1. 🤔 O Que É SVD? Quebrando o Conceito

Lembra quando falamos sobre **transformações lineares** no Módulo 6? A SVD é como se fosse uma "receita de bolo" para decompor qualquer matriz em três ingredientes fundamentais!

Matematicamente, para qualquer matriz $A$ de dimensões $m \times n$, a SVD nos dá:

$$A = U \Sigma V^T$$

Onde:
- $U$ é uma matriz $m \times m$ ortogonal (colunas são ortonormais)
- $\Sigma$ é uma matriz diagonal $m \times n$ com valores singulares
- $V^T$ é uma matriz $n \times n$ ortogonal transposta

### 🏠 Analogia da Casa

Pensa na sua matriz $A$ como uma casa:
- $U$: As **direções dos cômodos** (como os dados se espalham no espaço linha)
- $\Sigma$: A **importância de cada cômodo** (valores singulares em ordem decrescente)
- $V^T$: As **direções dos móveis** (como os dados se espalham no espaço coluna)

A magia acontece porque os valores em $\Sigma$ estão ordenados do maior para o menor. Os maiores valores capturam os padrões mais importantes!

**🎯 Dica do Pedro:** A SVD sempre existe para qualquer matriz! Diferente da decomposição em autovalores que só funciona para matrizes quadradas, a SVD é universal!

In [ ]:
# Vamos criar uma matriz simples para ver a SVD em ação
# Essa matriz representa dados com padrões claros
A = np.array([
    [1, 2, 3],
    [2, 4, 6],
    [1, 3, 5],
    [3, 6, 9]
])

print("🏠 Nossa matriz A (a 'casa' que vamos decompor):")
print(A)
print(f"\nDimensões: {A.shape[0]} linhas x {A.shape[1]} colunas")

# Aplicando SVD
U, sigma, Vt = np.linalg.svd(A)

print("\n🔍 Componentes da SVD:")
print(f"U (direções dos cômodos): {U.shape}")
print(f"Sigma (importância): {sigma.shape}")
print(f"V^T (direções dos móveis): {Vt.shape}")

print("\n📊 Valores singulares (em ordem decrescente):")
print(sigma)

## 2. 🧮 A Matemática por Trás da SVD

Tá, mas como diabos a SVD funciona matematicamente? Vou te explicar de forma descomplicada!

### 🔍 Os Valores Singulares

Os valores singulares $\sigma_i$ são as raízes quadradas dos autovalores de $A^T A$ (ou $A A^T$):

$$\sigma_i = \sqrt{\lambda_i(A^T A)}$$

### 🎯 As Matrizes U e V

- $U$: Contém os **vetores singulares à esquerda** (autovetores de $AA^T$)
- $V$: Contém os **vetores singulares à direita** (autovetores de $A^T A$)

### 🏗️ Propriedades Importantes

1. **Ortogonalidade**: $U^T U = I$ e $V^T V = I$
2. **Ordenação**: $\sigma_1 \geq \sigma_2 \geq ... \geq \sigma_r \geq 0$
3. **Posto**: O número de valores singulares não-nulos = posto da matriz

### 🧩 Reconstrução da Matriz

A matriz original pode ser reconstruída como:

$$A = \sum_{i=1}^{r} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

Onde $r$ é o posto da matriz. Cada termo $\sigma_i \mathbf{u}_i \mathbf{v}_i^T$ é uma matriz de posto 1!

**🎯 Dica do Pedro:** Pensa na SVD como uma "receita" que te diz exatamente como misturar padrões simples (matrizes de posto 1) para reconstruir seus dados complexos!

In [ ]:
# Vamos verificar as propriedades matemáticas da SVD
print("🔬 Verificando as propriedades matemáticas da SVD:\n")

# 1. Verificar se U^T * U = I (matriz identidade)
UtU = U.T @ U
print("1️⃣ U^T * U (deve ser próximo da identidade):")
print(np.round(UtU, 3))

# 2. Verificar se V^T * V = I
VtV = Vt @ Vt.T
print("\n2️⃣ V^T * V (deve ser próximo da identidade):")
print(np.round(VtV, 3))

# 3. Reconstruir a matriz original
# Primeiro, vamos criar a matriz Sigma completa
Sigma = np.zeros_like(A, dtype=float)
np.fill_diagonal(Sigma, sigma)

A_reconstruida = U @ Sigma @ Vt
print("\n3️⃣ Matriz original vs reconstruída:")
print("Original:")
print(A)
print("\nReconstruída:")
print(np.round(A_reconstruida, 3))

# 4. Erro de reconstrução
erro = np.linalg.norm(A - A_reconstruida)
print(f"\n4️⃣ Erro de reconstrução: {erro:.10f}")
print("Liiindo! Praticamente zero! 🎉")

## 3. 📊 Visualizando a SVD em Ação

Agora vamos ver como a SVD "enxerga" os dados! Vou criar um exemplo visual para mostrar como ela identifica as direções principais nos dados.

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/algebra-linear-para-ia-modulo-09_img_01.png)

In [ ]:
# Vamos criar dados 2D com padrão claro para visualizar a SVD
np.random.seed(42)

# Dados que têm uma direção principal clara
n_points = 100
x = np.random.randn(n_points)
y = 2 * x + 0.5 * np.random.randn(n_points)  # y correlacionado com x

# Matriz de dados (cada linha é um ponto)
dados = np.column_stack([x, y])

# Aplicar SVD
U_dados, sigma_dados, Vt_dados = np.linalg.svd(dados, full_matrices=False)

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Dados originais com direções principais
ax1.scatter(x, y, alpha=0.6, s=50)
ax1.set_title('Dados Originais com Direções Principais da SVD')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.grid(True, alpha=0.3)

# Plotar as direções principais (vetores singulares)
centro = np.mean(dados, axis=0)
escala = 3

# Primeira direção principal (maior valor singular)
v1 = Vt_dados[0] * sigma_dados[0] * escala
ax1.arrow(centro[0], centro[1], v1[0], v1[1], 
          head_width=0.3, head_length=0.2, fc='red', ec='red', linewidth=3,
          label=f'1ª direção (σ={sigma_dados[0]:.2f})')

# Segunda direção principal (menor valor singular)
v2 = Vt_dados[1] * sigma_dados[1] * escala
ax1.arrow(centro[0], centro[1], v2[0], v2[1], 
          head_width=0.3, head_length=0.2, fc='blue', ec='blue', linewidth=3,
          label=f'2ª direção (σ={sigma_dados[1]:.2f})')

ax1.legend()

# Gráfico 2: Valores singulares
ax2.bar(['σ₁', 'σ₂'], sigma_dados, color=['red', 'blue'], alpha=0.7)
ax2.set_title('Valores Singulares (Importância das Direções)')
ax2.set_ylabel('Valor Singular')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"🎯 A primeira direção captura {(sigma_dados[0]**2 / np.sum(sigma_dados**2) * 100):.1f}% da variância!")
print(f"🎯 A segunda direção captura {(sigma_dados[1]**2 / np.sum(sigma_dados**2) * 100):.1f}% da variância!")

## 4. 🎨 Compressão de Imagens com SVD

Agora vem a parte mais legal! Vamos usar SVD para comprimir imagens. É como se a gente tivesse um "Instagram filter" matemático que remove o que não é essencial!

### 🤔 Como Funciona?

1. **Decompomos** a imagem usando SVD: $A = U \Sigma V^T$
2. **Mantemos apenas** os $k$ maiores valores singulares
3. **Reconstruímos** usando: $A_k = \sum_{i=1}^{k} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$

### 📐 Taxa de Compressão

Para uma imagem $m \times n$, mantendo $k$ componentes:
- **Original**: $m \times n$ pixels
- **Comprimida**: $k(m + n + 1)$ valores
- **Taxa de compressão**: $\frac{mn}{k(m+n+1)}$

**🎯 Dica do Pedro:** A SVD é perfeita para imagens porque muitas vezes os primeiros valores singulares capturam a essência da imagem, e os demais são só "ruído"!

In [ ]:
# Vamos criar uma imagem sintética para demonstrar a compressão
# (Em projetos reais, você carregaria uma imagem real)

# Criando uma imagem sintética com padrões interessantes
x = np.linspace(0, 4*np.pi, 200)
y = np.linspace(0, 4*np.pi, 200)
X, Y = np.meshgrid(x, y)

# Imagem com padrões senoidais (simula texturas naturais)
imagem = np.sin(X) * np.cos(Y) + 0.5 * np.sin(2*X + Y) + 0.3 * np.sin(X - 2*Y)
# Normalizar para 0-255 (escala de cinza)
imagem = ((imagem - imagem.min()) / (imagem.max() - imagem.min()) * 255).astype(np.uint8)

print(f"📸 Imagem criada com dimensões: {imagem.shape}")
print(f"💾 Tamanho original: {imagem.size} pixels")

# Aplicar SVD na imagem
U_img, sigma_img, Vt_img = np.linalg.svd(imagem, full_matrices=False)

print(f"🔍 Número de valores singulares: {len(sigma_img)}")
print(f"🏆 Maior valor singular: {sigma_img[0]:.2f}")
print(f"🥉 Menor valor singular: {sigma_img[-1]:.2f}")

In [ ]:
# Função para reconstruir imagem com k componentes
def reconstruir_imagem_svd(U, sigma, Vt, k):
    """Reconstrói imagem usando apenas os k primeiros componentes SVD"""
    return (U[:, :k] @ np.diag(sigma[:k]) @ Vt[:k, :]).astype(np.uint8)

# Testando diferentes níveis de compressão
componentes = [1, 5, 10, 20, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, k in enumerate(componentes):
    # Reconstruir imagem
    img_comprimida = reconstruir_imagem_svd(U_img, sigma_img, Vt_img, k)
    
    # Calcular taxa de compressão
    m, n = imagem.shape
    tamanho_original = m * n
    tamanho_comprimido = k * (m + n + 1)
    taxa_compressao = tamanho_original / tamanho_comprimido
    
    # Calcular variância preservada
    variancia_preservada = np.sum(sigma_img[:k]**2) / np.sum(sigma_img**2) * 100
    
    # Plotar
    axes[i].imshow(img_comprimida, cmap='gray')
    axes[i].set_title(f'k={k} componentes\nCompressão: {taxa_compressao:.1f}x\nVariância: {variancia_preservada:.1f}%')
    axes[i].axis('off')

plt.suptitle('🎨 Compressão de Imagem usando SVD', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n🎯 Observe como:")
print("- Com poucos componentes, vemos os padrões principais")
print("- Conforme aumentamos k, mais detalhes aparecem")
print("- A compressão vs qualidade é um trade-off clássico!")

In [ ]:
# Vamos plotar como os valores singulares decaem
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Valores singulares
ax1.plot(sigma_img, linewidth=2, color='blue')
ax1.set_title('Decaimento dos Valores Singulares')
ax1.set_xlabel('Índice do Componente')
ax1.set_ylabel('Valor Singular')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Destacar alguns componentes importantes
componentes_destacados = [0, 4, 9, 19, 49]
ax1.scatter(componentes_destacados, sigma_img[componentes_destacados], 
           color='red', s=100, zorder=5)

# Gráfico 2: Variância acumulada
variancia_acumulada = np.cumsum(sigma_img**2) / np.sum(sigma_img**2) * 100
ax2.plot(variancia_acumulada, linewidth=2, color='green')
ax2.set_title('Variância Explicada Acumulada')
ax2.set_xlabel('Número de Componentes')
ax2.set_ylabel('Variância Explicada (%)')
ax2.grid(True, alpha=0.3)

# Linha de 90% de variância
idx_90 = np.where(variancia_acumulada >= 90)[0][0]
ax2.axhline(y=90, color='red', linestyle='--', alpha=0.7)
ax2.axvline(x=idx_90, color='red', linestyle='--', alpha=0.7)
ax2.text(idx_90+5, 85, f'90% com {idx_90+1} componentes', 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"💡 Para capturar 90% da variância, precisamos de apenas {idx_90+1} componentes!")
print(f"🎯 Isso representa uma compressão de {len(sigma_img)/(idx_90+1):.1f}x!")

## 5. 🎬 Sistema de Recomendação com SVD

Agora vamos para o Netflix da matemática! 🍿 A SVD é a base de muitos sistemas de recomendação, incluindo o famoso Netflix Prize!

### 🎯 Como Funciona?

1. **Matriz Usuário-Item**: Linhas = usuários, Colunas = filmes, Valores = avaliações
2. **Problema**: Matriz tem muitos valores faltando (nem todo mundo viu todos os filmes)
3. **Solução**: SVD encontra padrões latentes (gêneros, preferências)
4. **Predição**: Usa os padrões para prever avaliações faltantes

### 🧮 A Matemática

Se $R$ é nossa matriz de avaliações:

$$R \approx U_k \Sigma_k V_k^T$$

Onde:
- $U_k$: Preferências dos usuários por fatores latentes
- $\Sigma_k$: Importância de cada fator
- $V_k^T$: Como cada filme se relaciona com os fatores

**🎯 Dica do Pedro:** Os fatores latentes são como "gêneros abstratos" - podem ser ação, romance, sci-fi, ou combinações mais complexas que só a matemática consegue encontrar!

In [ ]:
# Vamos criar um dataset sintético de avaliações de filmes
np.random.seed(42)

# Parâmetros do sistema
n_usuarios = 50
n_filmes = 30
n_fatores_reais = 3  # Vamos simular 3 "gêneros" latentes

# Criar fatores latentes verdadeiros
# Usuários: preferências por cada fator (0-1)
usuarios_fatores = np.random.beta(2, 2, (n_usuarios, n_fatores_reais))

# Filmes: intensidade de cada fator (0-1)
filmes_fatores = np.random.beta(2, 2, (n_filmes, n_fatores_reais))

# Gerar avaliações baseadas nos fatores latentes
avaliacoes_completas = usuarios_fatores @ filmes_fatores.T

# Normalizar para escala 1-5 (como avaliações reais)
avaliacoes_completas = 1 + 4 * avaliacoes_completas

# Simular dados faltantes (nem todo mundo viu todos os filmes)
mascara_observada = np.random.random((n_usuarios, n_filmes)) < 0.3  # 30% de dados observados

# Matriz com dados faltantes (NaN onde não foi avaliado)
avaliacoes_observadas = avaliacoes_completas.copy()
avaliacoes_observadas[~mascara_observada] = np.nan

print(f"🎬 Sistema de Recomendação Criado!")
print(f"👥 Usuários: {n_usuarios}")
print(f"🎞️ Filmes: {n_filmes}")
print(f"📊 Avaliações observadas: {np.sum(mascara_observada)} de {n_usuarios * n_filmes} ({np.mean(mascara_observada)*100:.1f}%)")
print(f"🎭 Fatores latentes reais: {n_fatores_reais}")

In [ ]:
# Para aplicar SVD, precisamos lidar com valores faltantes
# Estratégia simples: substituir NaN pela média de cada filme

def preencher_com_media(matriz):
    """Preenche valores NaN com a média de cada coluna (filme)"""
    matriz_preenchida = matriz.copy()
    for j in range(matriz.shape[1]):
        coluna = matriz[:, j]
        media_coluna = np.nanmean(coluna)
        matriz_preenchida[np.isnan(coluna), j] = media_coluna
    return matriz_preenchida

# Preencher dados faltantes
avaliacoes_preenchidas = preencher_com_media(avaliacoes_observadas)

# Aplicar SVD
U_rec, sigma_rec, Vt_rec = np.linalg.svd(avaliacoes_preenchidas, full_matrices=False)

print("📈 SVD aplicada no sistema de recomendação!")
print(f"🔍 Valores singulares encontrados: {len(sigma_rec)}")

# Visualizar os valores singulares
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sigma_rec[:15], 'bo-', linewidth=2, markersize=8)
plt.title('Primeiros 15 Valores Singulares')
plt.xlabel('Componente')
plt.ylabel('Valor Singular')
plt.grid(True, alpha=0.3)

# Destacar os 3 primeiros (esperamos que capturem os fatores reais)
plt.scatter([0, 1, 2], sigma_rec[:3], color='red', s=100, zorder=5)
plt.text(0.5, sigma_rec[0]*0.9, '3 fatores\nlatentes?', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.subplot(1, 2, 2)
variancia_acum_rec = np.cumsum(sigma_rec**2) / np.sum(sigma_rec**2) * 100
plt.plot(variancia_acum_rec[:15], 'go-', linewidth=2, markersize=8)
plt.title('Variância Explicada Acumulada')
plt.xlabel('Número de Componentes')
plt.ylabel('Variância Explicada (%)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🎯 Os primeiros 3 componentes explicam {variancia_acum_rec[2]:.1f}% da variância!")
print(f"🎉 Como esperado, já que simulamos 3 fatores latentes!")

In [ ]:
# Fazer predições usando SVD com k=3 componentes
k = 3
avaliacoes_preditas = U_rec[:, :k] @ np.diag(sigma_rec[:k]) @ Vt_rec[:k, :]

# Avaliar qualidade das predições
# Vamos calcular erro apenas nos dados que foram "escondidos"
mascara_teste = ~mascara_observada  # Dados que o modelo não viu

# RMSE (Root Mean Square Error)
erro_quadratico = (avaliacoes_completas[mascara_teste] - avaliacoes_preditas[mascara_teste])**2
rmse = np.sqrt(np.mean(erro_quadratico))

print(f"🎯 Avaliação do Sistema de Recomendação:")
print(f"📊 RMSE: {rmse:.3f} (escala 1-5)")
print(f"📈 Erro médio: {np.mean(np.abs(avaliacoes_completas[mascara_teste] - avaliacoes_preditas[mascara_teste])):.3f}")

# Vamos ver alguns exemplos de predições
print("\n🔍 Exemplos de Predições vs Realidade:")
print("(Para filmes que os usuários não avaliaram originalmente)\n")

exemplos = np.where(mascara_teste)
for i in range(min(10, len(exemplos[0]))):
    usuario = exemplos[0][i]
    filme = exemplos[1][i]
    real = avaliacoes_completas[usuario, filme]
    predito = avaliacoes_preditas[usuario, filme]
    print(f"👤 Usuário {usuario:2d}, 🎬 Filme {filme:2d}: Real={real:.2f}, Predito={predito:.2f}, Erro={abs(real-predito):.2f}")

print(f"\n🎉 Liiindo! A SVD conseguiu capturar os padrões de preferência!")

## 6. 📊 Redução de Dimensionalidade: PCA vs SVD

Tá, mas qual a diferença entre SVD e PCA? Spoiler: eles são primos! 👨‍👩‍👧‍👦

### 🔗 A Conexão

O PCA (que vamos ver no próximo módulo) é basicamente SVD aplicada na matriz de covariância:

1. **PCA**: Encontra direções de máxima variância
2. **SVD**: Decompõe qualquer matriz em direções principais
3. **Conexão**: PCA pode ser calculado via SVD!

### 🎯 Quando Usar Cada Um?

- **SVD**: Dados faltantes, matrizes não-quadradas, sistemas de recomendação
- **PCA**: Redução de dimensionalidade clássica, análise de componentes principais

**🎯 Dica do Pedro:** SVD é mais geral e poderosa, PCA é mais específica para análise de variância. É como comparar um canivete suíço com uma chave de fenda especializada!

In [ ]:
# Vamos criar dados de alta dimensionalidade para demonstrar redução
# Simulando dados de sensores, pixels, features, etc.

np.random.seed(42)

# Criar dados com estrutura latente
n_amostras = 200
n_dimensoes = 50
n_fatores_latentes = 5

# Fatores latentes (variáveis não observadas que geram os dados)
fatores_latentes = np.random.randn(n_amostras, n_fatores_latentes)

# Matriz de carregamentos (como cada fator afeta cada dimensão)
carregamentos = np.random.randn(n_fatores_latentes, n_dimensoes)

# Gerar dados de alta dimensionalidade
dados_alta_dim = fatores_latentes @ carregamentos + 0.1 * np.random.randn(n_amostras, n_dimensoes)

print(f"📊 Dados de alta dimensionalidade criados:")
print(f"📏 Dimensões originais: {dados_alta_dim.shape}")
print(f"🎯 Fatores latentes reais: {n_fatores_latentes}")

# Centralizar os dados (importante para PCA/SVD)
dados_centralizados = dados_alta_dim - np.mean(dados_alta_dim, axis=0)

# Aplicar SVD
U_dim, sigma_dim, Vt_dim = np.linalg.svd(dados_centralizados, full_matrices=False)

# Calcular variância explicada por cada componente
variancia_explicada = (sigma_dim**2) / np.sum(sigma_dim**2) * 100
variancia_acumulada = np.cumsum(variancia_explicada)

# Visualizar
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Variância explicada por componente
ax1.bar(range(1, 16), variancia_explicada[:15], alpha=0.7, color='skyblue')
ax1.bar(range(1, 6), variancia_explicada[:5], alpha=0.9, color='red', 
        label='5 primeiros (fatores reais)')
ax1.set_title('Variância Explicada por Componente')
ax1.set_xlabel('Componente')
ax1.set_ylabel('Variância Explicada (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Variância acumulada
ax2.plot(range(1, 21), variancia_acumulada[:20], 'bo-', linewidth=2)
ax2.axhline(y=95, color='red', linestyle='--', alpha=0.7, label='95% da variância')
ax2.axvline(x=5, color='green', linestyle='--', alpha=0.7, label='5 componentes (real)')
ax2.set_title('Variância Explicada Acumulada')
ax2.set_xlabel('Número de Componentes')
ax2.set_ylabel('Variância Acumulada (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🎯 Os primeiros 5 componentes explicam {variancia_acumulada[4]:.1f}% da variância!")
print(f"🎉 Reduzimos de {n_dimensoes} para 5 dimensões mantendo quase toda informação!")

## 7. 🔥 Exercício Prático: Netflix Simplificado

Agora é sua vez! Vamos criar um mini-Netflix usando SVD! 🍿

**🎯 Seu Desafio:**
1. Complete o código para criar um sistema de recomendação
2. Teste diferentes números de componentes
3. Avalie a qualidade das predições
4. Faça recomendações para um usuário específico

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/algebra-linear-para-ia-modulo-09_img_02.png)

In [ ]:
# 🎬 EXERCÍCIO: Construa seu próprio Netflix!

# Dados do mini-Netflix
filmes = ['Ação 1', 'Ação 2', 'Romance 1', 'Romance 2', 'Comédia 1', 'Comédia 2', 'Terror 1', 'Terror 2']
usuarios = ['Alice', 'Bob', 'Carol', 'David', 'Eva']

# Matriz de avaliações (5 usuários x 8 filmes, escala 1-5, NaN = não assistiu)
avaliacoes = np.array([
    [5, 4, 1, 2, 3, 3, 1, 1],  # Alice: gosta de ação, não gosta de terror
    [4, 5, 2, 1, 4, 4, 2, 1],  # Bob: ação e comédia
    [1, 2, 5, 4, 2, 3, 1, 2],  # Carol: romance
    [2, 1, 4, 5, 1, 2, 4, 5],  # David: romance e terror
    [3, 4, 3, 2, 5, 4, 2, 1],  # Eva: comédia
], dtype=float)

# Simular dados faltantes (alguns filmes não foram assistidos)
np.random.seed(42)
mascara_faltantes = np.random.random(avaliacoes.shape) < 0.3  # 30% de dados faltantes
avaliacoes_com_faltantes = avaliacoes.copy()
avaliacoes_com_faltantes[mascara_faltantes] = np.nan

print("🎬 Mini-Netflix Dataset:")
print("\nAvaliações observadas (NaN = não assistiu):")
df_avaliacoes = pd.DataFrame(avaliacoes_com_faltantes, index=usuarios, columns=filmes)
print(df_avaliacoes)

# TODO: Complete as funções abaixo!

def preencher_dados_faltantes(matriz):
    """
    Preenche dados faltantes com a média de cada filme
    TODO: Implemente esta função!
    """
    # Sua implementação aqui
    matriz_preenchida = matriz.copy()
    for j in range(matriz.shape[1]):
        coluna = matriz[:, j]
        media = np.nanmean(coluna)
        matriz_preenchida[np.isnan(coluna), j] = media
    return matriz_preenchida

def aplicar_svd_recomendacao(matriz, k=2):
    """
    Aplica SVD e reconstrói matriz com k componentes
    TODO: Complete esta função!
    """
    # Preencher dados faltantes
    matriz_preenchida = preencher_dados_faltantes(matriz)
    
    # Aplicar SVD
    U, sigma, Vt = np.linalg.svd(matriz_preenchida, full_matrices=False)
    
    # Reconstruir com k componentes
    matriz_reconstruida = U[:, :k] @ np.diag(sigma[:k]) @ Vt[:k, :]
    
    return matriz_reconstruida, sigma

# Testar sua implementação
predicoes, valores_singulares = aplicar_svd_recomendacao(avaliacoes_com_faltantes, k=3)

print("\n🔮 Predições do sistema:")
df_predicoes = pd.DataFrame(np.round(predicoes, 2), index=usuarios, columns=filmes)
print(df_predicoes)

print(f"\n📊 Valores singulares: {np.round(valores_singulares, 2)}")

In [ ]:
# 🎯 PARTE 2 DO EXERCÍCIO: Análise e Recomendações

def recomendar_filmes(usuario_idx, avaliacoes_originais, predicoes, filmes, top_k=3):
    """
    Recommenda filmes que o usuário não assistiu
    TODO: Complete esta função!
    """
    # Identificar filmes não assistidos (eram NaN)
    nao_assistidos = np.isnan(avaliacoes_originais[usuario_idx])
    
    # Pegar predições para filmes não assistidos
    predicoes_nao_assistidos = predicoes[usuario_idx][nao_assistidos]
    filmes_nao_assistidos = np.array(filmes)[nao_assistidos]
    
    # Ordenar por predição (maior para menor)
    indices_ordenados = np.argsort(predicoes_nao_assistidos)[::-1]
    
    # Retornar top k recomendações
    top_filmes = filmes_nao_assistidos[indices_ordenados[:top_k]]
    top_predicoes = predicoes_nao_assistidos[indices_ordenados[:top_k]]
    
    return list(zip(top_filmes, top_predicoes))

# Testar recomendações para cada usuário
print("🎬 RECOMENDAÇÕES PERSONALIZADAS:")
print("=" * 50)

for i, usuario in enumerate(usuarios):
    recomendacoes = recomendar_filmes(i, avaliacoes_com_faltantes, predicoes, filmes)
    
    print(f"\n👤 {usuario}:")
    print(f"   Filmes já assistidos: {[filmes[j] for j in range(len(filmes)) if not np.isnan(avaliacoes_com_faltantes[i, j])]}")
    print(f"   🔥 Recomendações:")
    for j, (filme, predicao) in enumerate(recomendacoes, 1):
        print(f"      {j}. {filme} (predição: {predicao:.2f})")

# Visualizar matriz de correlação dos padrões encontrados
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.heatmap(avaliacoes, annot=True, cmap='RdYlBu_r', center=3, 
            xticklabels=filmes, yticklabels=usuarios, cbar_kws={'label': 'Avaliação'})
plt.title('Matriz Real (sem dados faltantes)')

plt.subplot(2, 2, 2)
sns.heatmap(predicoes, annot=True, fmt='.1f', cmap='RdYlBu_r', center=3,
            xticklabels=filmes, yticklabels=usuarios, cbar_kws={'label': 'Predição'})
plt.title('Predições SVD')

plt.subplot(2, 2, 3)
plt.bar(range(len(valores_singulares)), valores_singulares, alpha=0.7)
plt.title('Valores Singulares (Fatores Latentes)')
plt.xlabel('Componente')
plt.ylabel('Valor Singular')

plt.subplot(2, 2, 4)
# Erro por usuário
erros_usuario = []
for i in range(len(usuarios)):
    mask_observados = ~np.isnan(avaliacoes_com_faltantes[i])
    if np.any(mask_observados):
        erro = np.mean(np.abs(avaliacoes[i][mask_observados] - predicoes[i][mask_observados]))
        erros_usuario.append(erro)
    else:
        erros_usuario.append(0)

plt.bar(usuarios, erros_usuario, alpha=0.7, color='orange')
plt.title('Erro Médio por Usuário')
plt.ylabel('MAE (Mean Absolute Error)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print(f"\n🎉 Parabéns! Você criou seu próprio sistema de recomendação!")
print(f"📊 Erro médio geral: {np.mean(erros_usuario):.3f}")

## 8. 🚀 SVD Truncada: Otimizando para Big Data

Quando seus dados são ENORMES (tipo Instagram com bilhões de fotos), a SVD clássica pode ser lenta demais. Entra em cena a **SVD Truncada**! ⚡

### 🤔 O Que É?

Em vez de calcular TODOS os valores singulares, calculamos apenas os $k$ maiores:
- **SVD Completa**: $O(min(m^2n, mn^2))$ - muito lenta!
- **SVD Truncada**: $O(k \cdot min(m,n))$ - muito mais rápida!

### 🔧 Implementações Populares
- **scikit-learn**: `TruncatedSVD`
- **scipy**: `svds` (sparse SVD)
- **randomized SVD**: Para matrizes muito grandes

**🎯 Dica do Pedro:** Para dados reais, sempre use SVD truncada! É como pedir apenas as fatias mais gostosas da pizza em vez de comer a pizza inteira! 🍕

In [ ]:
# Comparando SVD completa vs truncada
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
import time

# Criar matriz grande para testar performance
np.random.seed(42)
m, n = 1000, 500
matriz_grande = np.random.randn(m, n)

print(f"🔥 Testando performance em matriz {m}x{n}")
print("=" * 50)

# 1. SVD completa (NumPy)
start_time = time.time()
U_completa, sigma_completa, Vt_completa = np.linalg.svd(matriz_grande, full_matrices=False)
tempo_completa = time.time() - start_time

print(f"⏱️ SVD Completa (NumPy): {tempo_completa:.3f}s")
print(f"📊 Componentes calculados: {len(sigma_completa)}")

# 2. SVD truncada (scikit-learn)
k = 50  # Queremos apenas 50 componentes
start_time = time.time()
svd_truncada = TruncatedSVD(n_components=k, random_state=42)
U_truncada = svd_truncada.fit_transform(matriz_grande)
sigma_truncada = svd_truncada.singular_values_
Vt_truncada = svd_truncada.components_
tempo_truncada = time.time() - start_time

print(f"⚡ SVD Truncada (sklearn): {tempo_truncada:.3f}s")
print(f"📊 Componentes calculados: {k}")
print(f"🚀 Speedup: {tempo_completa/tempo_truncada:.1f}x mais rápida!")

# 3. Comparar precisão dos valores singulares
erro_valores = np.abs(sigma_completa[:k] - sigma_truncada)
print(f"\n🎯 Erro médio nos valores singulares: {np.mean(erro_valores):.6f}")
print(f"📈 Erro máximo: {np.max(erro_valores):.6f}")

# Visualizar comparação
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Comparar valores singulares
ax1.plot(sigma_completa[:k], 'b-', linewidth=2, label='SVD Completa', alpha=0.7)
ax1.plot(sigma_truncada, 'r--', linewidth=2, label='SVD Truncada', alpha=0.7)
ax1.set_title('Comparação dos Valores Singulares')
ax1.set_xlabel('Componente')
ax1.set_ylabel('Valor Singular')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Comparar tempos
metodos = ['SVD\nCompleta', 'SVD\nTruncada']
tempos = [tempo_completa, tempo_truncada]
cores = ['blue', 'red']

bars = ax2.bar(metodos, tempos, color=cores, alpha=0.7)
ax2.set_title('Comparação de Performance')
ax2.set_ylabel('Tempo (segundos)')

# Adicionar valores nas barras
for bar, tempo in zip(bars, tempos):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{tempo:.3f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n💡 Variância explicada pelos {k} componentes: {svd_truncada.explained_variance_ratio_.sum()*100:.1f}%")

## 9. 🎭 Interpretando os Fatores Latentes

Uma das partes mais legais da SVD é tentar entender o que os fatores latentes representam! É como ser um detetive matemático! 🕵️‍♂️

### 🔍 O Que Procurar?

1. **Padrões nos vetores singulares**: Quais variáveis têm pesos altos juntas?
2. **Magnitude dos valores singulares**: Quão importante é cada fator?
3. **Interpretação do domínio**: O que faz sentido no contexto dos dados?

### 🎬 Exemplo: Sistema de Filmes
- **Fator 1**: Pode ser "Ação vs Romance"
- **Fator 2**: Pode ser "Mainstream vs Indie"
- **Fator 3**: Pode ser "Clássico vs Moderno"

**🎯 Dica do Pedro:** Os fatores latentes são como "DNA dos dados" - capturam a essência que nossos olhos não conseguem ver diretamente!

In [ ]:
# Vamos criar um exemplo mais elaborado para interpretar fatores latentes
# Simulando dados de avaliação de produtos com características conhecidas

np.random.seed(42)

# Produtos com características conhecidas
produtos = {
    'iPhone': [9, 2, 8, 9, 7],      # [tecnologia, preço_baixo, design, marca, durabilidade]
    'Samsung': [8, 3, 7, 7, 8],
    'Xiaomi': [7, 8, 6, 5, 6],
    'Motorola': [6, 7, 5, 6, 7],
    'Notebook_Gaming': [9, 1, 6, 8, 8],
    'Notebook_Basico': [4, 9, 4, 4, 5],
    'Tablet_Pro': [8, 2, 9, 8, 7],
    'Tablet_Kids': [3, 8, 5, 3, 6],
    'Smartwatch': [7, 4, 8, 7, 6],
    'Fones_Premium': [6, 2, 9, 9, 8]
}

caracteristicas = ['Tecnologia', 'Custo-Benefício', 'Design', 'Marca', 'Durabilidade']
nomes_produtos = list(produtos.keys())

# Converter para matriz
matriz_produtos = np.array(list(produtos.values()))

print("🛍️ Matriz de Características dos Produtos:")
df_produtos = pd.DataFrame(matriz_produtos, index=nomes_produtos, columns=caracteristicas)
print(df_produtos)

# Aplicar SVD
U_prod, sigma_prod, Vt_prod = np.linalg.svd(matriz_produtos, full_matrices=False)

print(f"\n🔍 Análise SVD:")
print(f"📊 Valores singulares: {np.round(sigma_prod, 2)}")

# Analisar os primeiros fatores latentes
n_fatores = 3

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Importância dos fatores
variancia_prod = (sigma_prod**2) / np.sum(sigma_prod**2) * 100
axes[0,0].bar(range(1, len(sigma_prod)+1), variancia_prod, alpha=0.7)
axes[0,0].set_title('Importância dos Fatores Latentes')
axes[0,0].set_xlabel('Fator')
axes[0,0].set_ylabel('Variância Explicada (%)')
axes[0,0].grid(True, alpha=0.3)

# 2. Primeiro fator latente (características)
fator1_caract = Vt_prod[0, :]
cores = ['red' if x > 0 else 'blue' for x in fator1_caract]
axes[0,1].bar(caracteristicas, fator1_caract, color=cores, alpha=0.7)
axes[0,1].set_title('Fator 1: Carregamentos das Características')
axes[0,1].set_ylabel('Carregamento')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)

# 3. Primeiro fator latente (produtos)
fator1_prod = U_prod[:, 0] * sigma_prod[0]
cores_prod = ['red' if x > 0 else 'blue' for x in fator1_prod]
axes[1,0].barh(nomes_produtos, fator1_prod, color=cores_prod, alpha=0.7)
axes[1,0].set_title('Fator 1: Scores dos Produtos')
axes[1,0].set_xlabel('Score do Fator')
axes[1,0].grid(True, alpha=0.3)

# 4. Visualização 2D dos primeiros dois fatores
fator1_prod = U_prod[:, 0] * sigma_prod[0]
fator2_prod = U_prod[:, 1] * sigma_prod[1]

axes[1,1].scatter(fator1_prod, fator2_prod, s=100, alpha=0.7)
for i, produto in enumerate(nomes_produtos):
    axes[1,1].annotate(produto, (fator1_prod[i], fator2_prod[i]), 
                      xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1,1].set_title('Produtos no Espaço dos 2 Primeiros Fatores')
axes[1,1].set_xlabel(f'Fator 1 ({variancia_prod[0]:.1f}% var)')
axes[1,1].set_ylabel(f'Fator 2 ({variancia_prod[1]:.1f}% var)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretação dos fatores
print("\n🎭 INTERPRETAÇÃO DOS FATORES LATENTES:")
print("=" * 50)

for i in range(min(3, len(sigma_prod))):
    print(f"\n🔍 FATOR {i+1} ({variancia_prod[i]:.1f}% da variância):")
    
    # Características mais importantes
    carregamentos = Vt_prod[i, :]
    indices_ordenados = np.argsort(np.abs(carregamentos))[::-1]
    
    print("   Características mais relevantes:")
    for j in indices_ordenados[:3]:
        sinal = '+' if carregamentos[j] > 0 else '-'
        print(f"   {sinal} {caracteristicas[j]}: {carregamentos[j]:.3f}")
    
    # Produtos extremos
    scores_produtos = U_prod[:, i] * sigma_prod[i]
    produto_max = np.argmax(scores_produtos)
    produto_min = np.argmin(scores_produtos)
    
    print(f"   Produto mais positivo: {nomes_produtos[produto_max]} ({scores_produtos[produto_max]:.3f})")
    print(f"   Produto mais negativo: {nomes_produtos[produto_min]} ({scores_produtos[produto_min]:.3f})")

print("\n🎉 Liiindo! Conseguimos descobrir os fatores que diferenciam os produtos!")

## 10. ⚡ Exercício Desafio: Análise de Sentimentos

Agora o desafio final! Vamos usar SVD para análise de sentimentos em reviews! 🎭

**🎯 Seu Objetivo:**
1. Criar uma matriz termo-documento
2. Aplicar SVD para encontrar tópicos latentes
3. Classificar sentimentos usando os fatores encontrados
4. Interpretar os tópicos descobertos

**🔧 Dicas:**
- Use bag-of-words ou TF-IDF
- Experimente diferentes números de componentes
- Visualize os tópicos encontrados

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/algebra-linear-para-ia-modulo-09_img_03.png)

In [ ]:
# 🎭 DESAFIO: Análise de Sentimentos com SVD

# Dataset sintético de reviews de filmes
reviews = {
    "Filme incrível, adorei a história e os personagens!": 1,  # Positivo
    "Excelente cinematografia, muito bem dirigido.": 1,
    "História emocionante, chorei no final.": 1,
    "Ótimos efeitos especiais, recomendo!": 1,
    "Perfeito para assistir com a família.": 1,
    "Filme horrível, história sem sentido.": 0,  # Negativo
    "Muito chato, quase dormi no cinema.": 0,
    "Péssimos atores, roteiro terrível.": 0,
    "Waste de tempo, não recomendo.": 0,
    "Boring e previsível demais.": 0,
    "Filme mediano, alguns momentos bons.": 1,  # Neutro/Positivo
    "História interessante mas execução fraca.": 0,  # Neutro/Negativo
}

textos = list(reviews.keys())
sentimentos = list(reviews.values())

print(f"📝 Dataset de Reviews:")
print(f"Total de reviews: {len(textos)}")
print(f"Positivos: {sum(sentimentos)}, Negativos: {len(sentimentos) - sum(sentimentos)}")

# TODO: Implemente as funções abaixo!

def criar_vocabulario(textos):
    """
    Cria vocabulário básico removendo pontuação e convertendo para minúsculas
    TODO: Complete esta função
    """
    import re
    vocabulario = set()
    
    for texto in textos:
        # Remover pontuação e converter para minúsculas
        palavras = re.findall(r'\b\w+\b', texto.lower())
        vocabulario.update(palavras)
    
    return sorted(list(vocabulario))

def criar_matriz_termo_documento(textos, vocabulario):
    """
    Cria matriz termo-documento (bag of words)
    TODO: Complete esta função
    """
    import re
    matriz = np.zeros((len(textos), len(vocabulario)))
    
    for i, texto in enumerate(textos):
        palavras = re.findall(r'\b\w+\b', texto.lower())
        for palavra in palavras:
            if palavra in vocabulario:
                j = vocabulario.index(palavra)
                matriz[i, j] += 1
    
    return matriz

# Criar representação vetorial dos textos
vocab = criar_vocabulario(textos)
matriz_docs = criar_matriz_termo_documento(textos, vocab)

print(f"\n🔤 Vocabulário criado:")
print(f"Tamanho do vocabulário: {len(vocab)}")
print(f"Primeiras 10 palavras: {vocab[:10]}")
print(f"\n📊 Matriz termo-documento: {matriz_docs.shape}")
print(f"Densidade: {np.count_nonzero(matriz_docs) / matriz_docs.size * 100:.1f}% (palavras únicas)")

# Aplicar SVD
U_text, sigma_text, Vt_text = np.linalg.svd(matriz_docs, full_matrices=False)

print(f"\n🔍 SVD aplicada:")
print(f"Componentes: {len(sigma_text)}")
print(f"Valores singulares: {np.round(sigma_text[:5], 2)}")

In [ ]:
# Análise dos tópicos encontrados pela SVD

def analisar_topicos_svd(Vt, vocabulario, n_topicos=3, n_palavras=5):
    """
    Analisa os tópicos latentes encontrados pela SVD
    TODO: Complete esta função
    """
    print("🎭 TÓPICOS LATENTES DESCOBERTOS PELA SVD:")
    print("=" * 50)
    
    for i in range(min(n_topicos, Vt.shape[0])):
        print(f"\n📋 Tópico {i+1}:")
        
        # Palavras com maior peso positivo
        carregamentos = Vt[i, :]
        indices_pos = np.argsort(carregamentos)[::-1][:n_palavras]
        indices_neg = np.argsort(carregamentos)[:n_palavras]
        
        print("   Palavras mais positivas:")
        for idx in indices_pos:
            if carregamentos[idx] > 0:
                print(f"   + {vocabulario[idx]}: {carregamentos[idx]:.3f}")
        
        print("   Palavras mais negativas:")
        for idx in indices_neg:
            if carregamentos[idx] < 0:
                print(f"   - {vocabulario[idx]}: {carregamentos[idx]:.3f}")

# Analisar tópicos
analisar_topicos_svd(Vt_text, vocab, n_topicos=3)

# Projetar documentos no espaço latente
n_componentes = 2
documentos_projetados = U_text[:, :n_componentes] @ np.diag(sigma_text[:n_componentes])

# Visualizar documentos no espaço latente
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot dos documentos
cores = ['red' if s == 0 else 'blue' for s in sentimentos]
labels = ['Negativo' if s == 0 else 'Positivo' for s in sentimentos]

scatter = ax1.scatter(documentos_projetados[:, 0], documentos_projetados[:, 1], 
                     c=cores, s=100, alpha=0.7)

# Adicionar labels aos pontos
for i, (x, y) in enumerate(documentos_projetados):
    ax1.annotate(f'Doc{i+1}', (x, y), xytext=(5, 5), 
                textcoords='offset points', fontsize=8)

ax1.set_title('Documentos no Espaço Latente SVD')
ax1.set_xlabel('Componente 1 (Tópico Principal)')
ax1.set_ylabel('Componente 2 (Segundo Tópico)')
ax1.grid(True, alpha=0.3)

# Criar legenda manual
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', label='Negativo'),
                  Patch(facecolor='blue', label='Positivo')]
ax1.legend(handles=legend_elements)

# Importância dos componentes
variancia_text = (sigma_text**2) / np.sum(sigma_text**2) * 100
ax2.bar(range(1, min(8, len(sigma_text)+1)), variancia_text[:min(7, len(variancia_text))], 
        alpha=0.7, color='green')
ax2.set_title('Importância dos Tópicos Latentes')
ax2.set_xlabel('Tópico')
ax2.set_ylabel('Variância Explicada (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Avaliar separação dos sentimentos
print(f"\n🎯 ANÁLISE DE SEPARAÇÃO:")
media_pos = np.mean(documentos_projetados[np.array(sentimentos) == 1], axis=0)
media_neg = np.mean(documentos_projetados[np.array(sentimentos) == 0], axis=0)

print(f"📊 Centro dos positivos: [{media_pos[0]:.3f}, {media_pos[1]:.3f}]")
print(f"📊 Centro dos negativos: [{media_neg[0]:.3f}, {media_neg[1]:.3f}]")
print(f"📏 Distância entre centros: {np.linalg.norm(media_pos - media_neg):.3f}")

if np.linalg.norm(media_pos - media_neg) > 1:
    print("🎉 Excelente! SVD conseguiu separar bem os sentimentos!")
else:
    print("🤔 SVD teve dificuldade em separar os sentimentos neste dataset simples.")

print(f"\n💡 O primeiro componente explica {variancia_text[0]:.1f}% da variância!")
print(f"💡 Os dois primeiros explicam {variancia_text[0] + variancia_text[1]:.1f}% da variância!")

## 11. 🔄 SVD vs PCA vs NMF: O Confronto dos Gigantes

Antes de encerrar, vamos comparar a SVD com outras técnicas de decomposição! É como um UFC da matemática! 🥊

### 🥊 SVD vs PCA vs NMF

| Técnica | Quando Usar | Vantagens | Desvantagens |
|---------|-------------|-----------|-------------|
| **SVD** | Qualquer matriz, sistemas de recomendação | Universal, precisa | Pode ser lenta |
| **PCA** | Redução de dimensionalidade, análise de variância | Interpretável, eficiente | Só matrizes quadradas de covariância |
| **NMF** | Dados não-negativos, tópicos em texto | Componentes interpretáveis | Só dados ≥ 0 |

### 🎯 Conexões Matemáticas

- **PCA ≈ SVD** da matriz de covariância
- **SVD** é mais geral que PCA
- **NMF** força positividade nos componentes

**🎯 Dica do Pedro:** SVD é como um canivete suíço - resolve quase tudo. PCA é especialista em variância. NMF é perfeito quando você quer componentes que fazem sentido intuitivo (como ingredientes de uma receita)!

In [ ]:
# Comparação prática: SVD vs PCA vs NMF
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import StandardScaler

# Criar dados teste (imagem simples)
np.random.seed(42)
n_pixels = 100
imagem_teste = np.random.exponential(2, (n_pixels, n_pixels))  # Dados não-negativos

# Adicionar alguns padrões estruturados
x, y = np.meshgrid(np.linspace(0, 10, n_pixels), np.linspace(0, 10, n_pixels))
padrao1 = 5 * np.exp(-((x-3)**2 + (y-3)**2) / 4)  # Gaussiana
padrao2 = 3 * np.exp(-((x-7)**2 + (y-7)**2) / 6)  # Outra gaussiana
imagem_teste += padrao1 + padrao2

print(f"🖼️ Imagem teste criada: {imagem_teste.shape}")
print(f"📊 Min: {imagem_teste.min():.2f}, Max: {imagem_teste.max():.2f}")

# Aplicar as três técnicas
n_components = 10

# 1. SVD
start_time = time.time()
U_comp, sigma_comp, Vt_comp = np.linalg.svd(imagem_teste, full_matrices=False)
imagem_svd = U_comp[:, :n_components] @ np.diag(sigma_comp[:n_components]) @ Vt_comp[:n_components, :]
tempo_svd = time.time() - start_time

# 2. PCA (aplicado nas linhas da imagem)
start_time = time.time()
pca = PCA(n_components=n_components)
scaler = StandardScaler()
imagem_scaled = scaler.fit_transform(imagem_teste)
componentes_pca = pca.fit_transform(imagem_scaled)
imagem_pca = scaler.inverse_transform(pca.inverse_transform(componentes_pca))
tempo_pca = time.time() - start_time

# 3. NMF (Non-negative Matrix Factorization)
start_time = time.time()
nmf = NMF(n_components=n_components, random_state=42, max_iter=1000)
componentes_nmf = nmf.fit_transform(imagem_teste)
imagem_nmf = componentes_nmf @ nmf.components_
tempo_nmf = time.time() - start_time

# Calcular erros de reconstrução
erro_svd = np.linalg.norm(imagem_teste - imagem_svd)
erro_pca = np.linalg.norm(imagem_teste - imagem_pca)
erro_nmf = np.linalg.norm(imagem_teste - imagem_nmf)

print(f"\n⚡ COMPARAÇÃO DE PERFORMANCE:")
print(f"{'Método':<8} {'Tempo (ms)':<12} {'Erro Recons.':<15} {'RMSE':<10}")
print("-" * 50)
print(f"{'SVD':<8} {tempo_svd*1000:<12.1f} {erro_svd:<15.3f} {erro_svd/np.sqrt(imagem_teste.size):<10.4f}")
print(f"{'PCA':<8} {tempo_pca*1000:<12.1f} {erro_pca:<15.3f} {erro_pca/np.sqrt(imagem_teste.size):<10.4f}")
print(f"{'NMF':<8} {tempo_nmf*1000:<12.1f} {erro_nmf:<15.3f} {erro_nmf/np.sqrt(imagem_teste.size):<10.4f}")

# Visualizar resultados
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Original
im1 = axes[0,0].imshow(imagem_teste, cmap='viridis')
axes[0,0].set_title('Original')
axes[0,0].axis('off')
plt.colorbar(im1, ax=axes[0,0], fraction=0.046, pad=0.04)

# SVD
im2 = axes[0,1].imshow(imagem_svd, cmap='viridis')
axes[0,1].set_title(f'SVD (erro: {erro_svd:.1f})')
axes[0,1].axis('off')
plt.colorbar(im2, ax=axes[0,1], fraction=0.046, pad=0.04)

# PCA
im3 = axes[1,0].imshow(imagem_pca, cmap='viridis')
axes[1,0].set_title(f'PCA (erro: {erro_pca:.1f})')
axes[1,0].axis('off')
plt.colorbar(im3, ax=axes[1,0], fraction=0.046, pad=0.04)

# NMF
im4 = axes[1,1].imshow(imagem_nmf, cmap='viridis')
axes[1,1].set_title(f'NMF (erro: {erro_nmf:.1f})')
axes[1,1].axis('off')
plt.colorbar(im4, ax=axes[1,1], fraction=0.046, pad=0.04)

plt.suptitle(f'Comparação: SVD vs PCA vs NMF ({n_components} componentes)', fontsize=14)
plt.tight_layout()
plt.show()

# Comparar variância explicada
var_svd = np.sum(sigma_comp[:n_components]**2) / np.sum(sigma_comp**2) * 100
var_pca = np.sum(pca.explained_variance_ratio_) * 100
var_nmf = 100 - (erro_nmf**2 / np.linalg.norm(imagem_teste)**2) * 100

print(f"\n📊 VARIÂNCIA EXPLICADA:")
print(f"SVD: {var_svd:.1f}%")
print(f"PCA: {var_pca:.1f}%")
print(f"NMF: {var_nmf:.1f}% (aproximado)")

print(f"\n🎯 Vencedor em precisão: {'SVD' if erro_svd <= min(erro_pca, erro_nmf) else 'PCA' if erro_pca <= erro_nmf else 'NMF'}")
print(f"🏃 Vencedor em velocidade: {'SVD' if tempo_svd <= min(tempo_pca, tempo_nmf) else 'PCA' if tempo_pca <= tempo_nmf else 'NMF'}")

## 12. 🎓 Resumo: O Que Aprendemos Sobre SVD

Parabéns! Você completou uma jornada épica pela SVD! 🎉 Vamos recapitular os pontos principais:

### 🧮 **Fundamentos Matemáticos**
- SVD decompõe qualquer matriz: $A = U \Sigma V^T$
- $U$: direções no espaço das linhas
- $\Sigma$: importância de cada direção (valores singulares)
- $V^T$: direções no espaço das colunas

### 🎨 **Aplicações Práticas**
- **Compressão de Imagens**: Mantém qualidade com menos dados
- **Sistemas de Recomendação**: Encontra padrões de preferência
- **Redução de Dimensionalidade**: Remove ruído, mantém informação
- **Análise de Texto**: Descobre tópicos latentes

### ⚡ **Otimizações**
- **SVD Truncada**: Para datasets grandes
- **Randomized SVD**: Para matrizes enormes
- **Sparse SVD**: Para dados esparsos

### 🎯 **Dicas Importantes**
1. SVD sempre existe (diferente de eigendecomposition)
2. Valores singulares em ordem decrescente
3. Primeiros componentes capturam padrões principais
4. Trade-off entre compressão e qualidade

### 🔮 **Próximos Passos**
No próximo módulo, vamos mergulhar em **Autovetores e Autovalores** - os primos da SVD que são essenciais para PCA!

**🎯 Dica Final do Pedro:** A SVD é como um "raio-X matemático" - ela revela a estrutura interna dos seus dados que você não consegue ver a olho nu. Use com sabedoria e sempre experimente diferentes números de componentes!

### 📚 **Para Estudar Mais**
- Randomized SVD para Big Data
- SVD em sistemas de recomendação reais
- Conexões entre SVD e outras decomposições
- SVD para processamento de sinais

Liiindo! Agora você é um expert em SVD! 🚀✨

In [ ]:
# 🎊 PARABÉNS! Código de celebração!
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(12, 8))

# Criar visualização de celebração
ax.text(0.5, 0.7, '🎉 PARABÉNS! 🎉', fontsize=40, ha='center', va='center', 
        transform=ax.transAxes, weight='bold')

ax.text(0.5, 0.5, 'Você dominou a SVD!', fontsize=24, ha='center', va='center',
        transform=ax.transAxes, style='italic')

# Estatísticas do notebook
conceitos_aprendidos = [
    '✅ Decomposição SVD matemática',
    '✅ Compressão de imagens',
    '✅ Sistemas de recomendação', 
    '✅ Redução de dimensionalidade',
    '✅ Análise de sentimentos',
    '✅ Interpretação de fatores latentes',
    '✅ SVD vs PCA vs NMF',
    '✅ Otimizações e truques'
]

y_start = 0.35
for i, conceito in enumerate(conceitos_aprendidos):
    ax.text(0.1, y_start - i*0.03, conceito, fontsize=12, 
            transform=ax.transAxes, va='center')

# Caixa de destaque
bbox = FancyBboxPatch((0.05, 0.05), 0.9, 0.4, 
                      boxstyle="round,pad=0.02", 
                      facecolor='lightblue', 
                      edgecolor='navy', 
                      alpha=0.3,
                      transform=ax.transAxes)
ax.add_patch(bbox)

ax.text(0.5, 0.25, '🚀 Próximo Destino: Autovetores e Autovalores! 🚀', 
        fontsize=16, ha='center', va='center', transform=ax.transAxes, 
        weight='bold', color='navy')

ax.text(0.5, 0.15, 'Você está preparado para descobrir os eixos de rotação da IA!', 
        fontsize=14, ha='center', va='center', transform=ax.transAxes, 
        style='italic', color='navy')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("🎓 MÓDULO 9 CONCLUÍDO COM SUCESSO! 🎓")
print("="*60)
print("📊 Você implementou:")
print("   • Sistema de recomendação completo")
print("   • Compressor de imagens")
print("   • Analisador de sentimentos")
print("   • Comparador de técnicas de decomposição")
print("\n🏆 Conquistas desbloqueadas:")
print("   🥇 Mestre da Decomposição SVD")
print("   🥈 Expert em Sistemas de Recomendação")
print("   🥉 Especialista em Compressão de Dados")
print("\n🎯 Próxima aventura: Autovetores e Autovalores!")
print("📚 Continue sua jornada na Álgebra Linear para IA!")
print("\n" + "="*60)
print("Até o próximo módulo! 👋")
print("- Pedro Guth")